### Algoritmos Geneticos para solucionar o problema do escalonamento de horários

In [1]:
#Import .ipynb as module: %run name.ipynb
import math
import random
import copy

In [2]:
### Data Structure for the Program Schedule Problem ###

# Macros
WEEK_SIZE         = 5
WEEK_CLASSES_SIZE = 10

# Class Teacher
class Teacher:
    _id = 0
    def __init__(self, name):
        Teacher._id  = Teacher._id +1
        self.id      = Teacher._id
        self.name    = name
        
    def __repr__(self):
        return 'Teacher {}: {}'.format(self.id, self.name)
    
# Class Course
class Course:
    _id = 0
    def __init__(self, name, classes, semester, teacher = None):
        Course._id      = Course._id +1
        self.id         = Course._id
        self.name       = name
        self.classes    = classes
        self.ammo       = classes
        self.semester   = semester
        self.teacher    = teacher
        
    def setTeacher(self, teacher):
        self.teacher = teacher
        
    def __repr__(self):
        if self.teacher is not None:
            return 'Course {}: {} ({} classes, {}o semester, {} teacher.id)'.format(self.id, self.name, self.classes, self.semester, self.teacher.id)
        else:
            return 'Course {}: {} ({} classes, {}o semester)'.format(self.id, self.name, self.classes, self.semester)
        
# Class Program
class Program:
    def __init__(self, name, semesters):
        self.checked        = False
        self.name           = name
        self.semesters      = semesters
        self.courses        = []
        self.teachers       = []
        self.tAccu          = []
        self.tCollisionMax  = 0
        
    # Utils
    def getCoursesBySemester(self, semester):
        coursesSemester = []
        for i in self.courses:
            if(i.semester is semester and i.ammo > 0):
                coursesSemester.append(i)
        return coursesSemester
    
    def reloadCoursesAmmo(self):
        for i in self.courses:
            i.ammo = i.classes
    
    def getTeacherByName(self, name):
        for teacher in self.teachers:
            if(teacher.name == name):
                return teacher
        return None
    
    def checkCourses(self):
        for course in self.courses:
            if(course.teacher is None):
                print('Info: Courses information are incomplete')
                return False
        print('Info: Courses information are complete')
        self.tAccu = list([0 for i in range(len(self.teachers))])
        
        for course in self.courses:
            self.tAccu[course.teacher.id-1]+=1
        
        for tAccu_ in self.tAccu:
            if(tAccu_>=2):
                self.tCollisionMax += ( math.factorial(tAccu_)/(math.factorial(tAccu_-2)*2) )#Cn,2 = n!/((n-2)!
                
        return True
    
    # Add/Remove
    def addCourse(self, course):
        try:
            if(course not in self.courses):
                if(sum([c.classes for c in self.getCoursesBySemester(course.semester)])+course.classes<=WEEK_CLASSES_SIZE):
                    self.courses.append(course)
        except:
            return
    
    def removeCourse(self, course):
        try:
            self.courses.remove(course)
        except:
            return
    
    def addTeacher(self, teacher):
        try:
            if(teacher not in self.teachers):
                self.teachers.append(teacher)
        except:
            return
    
    def removeTeacher(self, teacher):
        try:
            self.teachers.remove(teacher)
        except:
            return
        
    def __repr__(self):
        return 'Program: {} ({} semesters, {} courses, {} teachers)'.format(self.name, self.semesters, len(self.courses), len(self.teachers))

In [3]:
### --- Input Data --- ###
### Program of EngComp ###
engComp = Program('EngComp', 10)

### Teachers ###
teachers = []
teachers.append(Teacher('Maria Eugenia'))
teachers.append(Teacher('Joacillo'))
teachers.append(Teacher('Roberto Carlos'))
teachers.append(Teacher('Bento'))
teachers.append(Teacher('Murilo'))
teachers.append(Teacher('Parente'))
teachers.append(Teacher('Alisson Linhares'))
teachers.append(Teacher('Glauber'))
teachers.append(Teacher('Ernani'))
teachers.append(Teacher('Anaxágoras'))
teachers.append(Teacher('George Sales'))
teachers.append(Teacher('Ronaldo'))
teachers.append(Teacher('Serra'))
teachers.append(Teacher('Fernando Parente'))
teachers.append(Teacher('Francisco Jose'))
teachers.append(Teacher('Cristiane Borges'))
teachers.append(Teacher('Nidia'))
teachers.append(Teacher('Cesar Olavo'))
teachers.append(Teacher('Elias'))
teachers.append(Teacher('Hairon'))
teachers.append(Teacher('Paulo Regis'))
teachers.append(Teacher('Cidcley'))
teachers.append(Teacher('Pedrosa'))
### Put teachers on program
for teacher in teachers:
    engComp.addTeacher(teacher)
    
### Courses ###
courses = []
# 1 Semester
courses.append(Course('Lógica Matemática'                             ,2,1, engComp.getTeacherByName('Maria Eugenia')))
courses.append(Course('Introdução à Programação'                      ,3,1, engComp.getTeacherByName('Ernani')))
courses.append(Course('Eletrônica Digital'                            ,3,1, engComp.getTeacherByName('Joacillo')))
courses.append(Course('Cálculo I'                                     ,2,1, engComp.getTeacherByName('Roberto Carlos')))
# 2 Semester
courses.append(Course('Matemática Discreta'                           ,2,2, engComp.getTeacherByName('Murilo')))
courses.append(Course('Programação Orientada à Objetos'               ,2,2, engComp.getTeacherByName('Alisson Linhares')))
courses.append(Course('Eletrônica Analógica'                          ,2,2, engComp.getTeacherByName('Bento')))
courses.append(Course('Cálculo II'                                    ,2,2, engComp.getTeacherByName('Roberto Carlos')))
courses.append(Course('Física-Eletricidade'                           ,2,2, engComp.getTeacherByName('Parente')))
# 3 Semester
courses.append(Course('Introdução à Análise de Algoritmos'            ,2,3, engComp.getTeacherByName('Glauber')))
courses.append(Course('Estrutura de Dados'                            ,2,3, engComp.getTeacherByName('Ernani')))
courses.append(Course('Circuitos Eletrônicos'                         ,2,3, engComp.getTeacherByName('Bento')))
courses.append(Course('Arquitetura de Computadores'                   ,2,3, engComp.getTeacherByName('Alisson Linhares')))
courses.append(Course('Física-Eletromagnetismo'                       ,2,3, engComp.getTeacherByName('George Sales')))
# 4 Semester
courses.append(Course('Aspectos Teóricos da Computação'               ,2,4, engComp.getTeacherByName('Ernani')))
courses.append(Course('Pesquisa e Ordenação'                          ,2,4, engComp.getTeacherByName('Ronaldo')))
courses.append(Course('Microcontroladores e Microprocessadores'       ,3,4, engComp.getTeacherByName('Anaxágoras')))
courses.append(Course('Geometria Analítica e Algebra Linear'          ,3,4, engComp.getTeacherByName('Murilo')))
# 5 Semester
courses.append(Course('Metodologia Científica e Tecnológica'          ,1,5, engComp.getTeacherByName('Cesar Olavo')))
courses.append(Course('Cálculo Numérico'                              ,2,5, engComp.getTeacherByName('Glauber')))
courses.append(Course('Banco de Dados'                                ,2,5, engComp.getTeacherByName('Serra')))
courses.append(Course('Sistemas Lineares'                             ,2,5, engComp.getTeacherByName('Francisco Jose')))
courses.append(Course('Sistemas Operacionais'                         ,2,5, engComp.getTeacherByName('Fernando Parente')))
# 6 Semester
courses.append(Course('Engenharia de Software'                        ,2,6, engComp.getTeacherByName('Cesar Olavo')))
courses.append(Course('Probabilidade e Estatística'                   ,2,6, engComp.getTeacherByName('Maria Eugenia')))
courses.append(Course('Redes de Computadores'                         ,2,6, engComp.getTeacherByName('Nidia')))
courses.append(Course('Sistemas Embarcados'                           ,3,6, engComp.getTeacherByName('Elias')))
# 7 Semester
courses.append(Course('Interação Humano-Computador'                   ,2,7, engComp.getTeacherByName('Hairon')))
courses.append(Course('Computação Gráfica'                            ,2,7, engComp.getTeacherByName('Nidia')))
courses.append(Course('Grafos'                                        ,2,7, engComp.getTeacherByName('Glauber')))
courses.append(Course('Produção Textual'                              ,1,7, engComp.getTeacherByName('Cristiane Borges')))
courses.append(Course('Introdução à Automação Industrial e Controle'  ,2,7, engComp.getTeacherByName('Joacillo')))
# 8 Semester
courses.append(Course('Projeto de Sistemas de Informação'             ,2,8, engComp.getTeacherByName('Hairon')))
courses.append(Course('Inteligência Computacional'                    ,2,8, engComp.getTeacherByName('Ronaldo')))
courses.append(Course('Sistemas Distribuídos'                         ,1,8, engComp.getTeacherByName('Cidcley')))
courses.append(Course('Sistemas de Tempo Real'                        ,1,8, engComp.getTeacherByName('Paulo Regis')))
courses.append(Course('Aplicações de Controle'                        ,2,8, engComp.getTeacherByName('Paulo Regis')))
# 9 Semester
courses.append(Course('Trabalho de Graduação Interdisciplinar'        ,1,9, engComp.getTeacherByName('Cidcley')))
courses.append(Course('Empreendedorismo e Gestão'                     ,1,9, engComp.getTeacherByName('Serra')))
courses.append(Course('Programação Paralela e Distribuída'            ,3,9, engComp.getTeacherByName('Cidcley')))
courses.append(Course('Optativa 1'                                    ,2,9, engComp.getTeacherByName('Pedrosa')))
# 10 Semester
courses.append(Course('Ética e Filosofia'                            ,1,10, engComp.getTeacherByName('Bento')))
courses.append(Course('Projeto Social'                               ,1,10, engComp.getTeacherByName('Cristiane Borges')))
courses.append(Course('Optativa 2'                                   ,2,10, engComp.getTeacherByName('Pedrosa')))
courses.append(Course('Optativa 3'                                   ,2,10, engComp.getTeacherByName('Ronaldo')))
### Put courses on program
for course in courses:
    engComp.addCourse(course)
    
### Print program object
print(engComp)
engComp.checkCourses()

Program: EngComp (10 semesters, 45 courses, 23 teachers)
Info: Courses information are complete


True

In [4]:
class Schedule:
    def __init__(self, program):
        # week name list
        self.weekName = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex']
        # schedule 3-dimensional list
        self.schedule = list([[[None for z in range((int)(WEEK_CLASSES_SIZE/WEEK_SIZE))] for j in range(WEEK_SIZE)] for i in range(program.semesters)])
        # generate random schedule based on the program (engComp)
        self.generateRandomSchedule(program)
        # reload course ammo to generate another schedule based on the program (engComp)
        program.reloadCoursesAmmo()
    
    def __getitem__(self, key):
        return self.schedule[key]
    
    def __len__(self):
        return len(self.schedule)
    
    # method to generate a random schedule based on courses of the program (engComp)
    def generateRandomSchedule(self, program):
        for semester in range(1, program.semesters+1):
            history = []
            while(True):
                courses = program.getCoursesBySemester(semester)
                if(courses == []):
                    break
                randomCourse = random.choice(courses)
                program.courses[program.courses.index(randomCourse)].ammo-=1
                #Put class on a random place
                while(True):
                    random1 = random.randint(1,WEEK_SIZE)
                    random2 = random.randint(1,(int)(WEEK_CLASSES_SIZE/WEEK_SIZE))
                    if((random1,random2) not in history):
                        history.append((random1,random2))
                        break
                #Put random choosen course
                self.schedule[semester-1][random1-1][random2-1] = randomCourse
        return True
    
    # toString equivalent method
    def __repr__(self):
        strOut = '|  '
        for i in self.weekName:
            strOut += i + '  '
        strOut += '|\n'
        for i in range(len(self.schedule)):
            strOut += '|---------------------------|\n'
            if(i+1>=10):
                strOut += '|        ' + 'Semester ' + str(i+1) + '        |\n'
            else:
                strOut += '|        ' + 'Semester ' + str(i+1) + '         |\n'
            strOut += '|---------------------------|\n'
            strOut += '|  '
            for j in range(len(self.schedule[i])):
                if(self.schedule[i][j][0] is None):
                    strOut += ' ' + str(0) + '   '
                elif(self.schedule[i][j][0].id<10):
                    strOut += ' ' + str(self.schedule[i][j][0].id) + '   '
                else: 
                    strOut += ''  + str(self.schedule[i][j][0].id) + '   '
            strOut += '| AB\n'
            strOut += '|  '
            for j in range(len(self.schedule[i])):
                if(self.schedule[i][j][1] is None):
                    strOut += ' ' + str(0) + '   '
                elif(self.schedule[i][j][1].id<10):
                    strOut += ' ' + str(self.schedule[i][j][1].id) + '   '
                else: 
                    strOut += ''  + str(self.schedule[i][j][1].id) + '   '
            strOut += '| CD\n'
        return strOut

In [5]:
class GeneticAlgorithm_ProgramSchedule:
    #|------------------------------- PARAMETERS --------------------------------|
    #| maxIterations    -> Maximum amount of iterations                          |
    #| populationSize   -> Initial population size                               |
    #| geneShift        -> Genome mutation rule                                  |
    #| reproductionRate -> Rate for number of couples used at cross-over         |
    #|---------------------------------------------------------------------------|
    def __init__(self, maxIterations, populationSize, reproductionRate=0.5):
        self.maxIterations     = maxIterations
        self.populationSize    = populationSize
        self.couplesSize       = int(self.populationSize*reproductionRate)
        self.fitnessResults    = list()
        
    def geneticRun(self, program):
        ### Initial Population
        schedulesPopulation = list([Schedule(program) for i in range(self.populationSize)])
        
        ### Iterations
        for iteration in range(1,self.maxIterations+1):
            ### Goal Test [1]
            goalResult = self.goalTest(program, schedulesPopulation)
            if(goalResult!=-1):
                return goalResult,'Genetic Algorithm Succeeded (Gen {})'.format(iteration)
            
            ### Evolution (Selection, Cross-Over and Mutation)
            schedulesPopulation_new = list()
            
            ### Selection
            selectedBreeders = self.selection(schedulesPopulation)
            males   = selectedBreeders[:self.couplesSize]
            females = selectedBreeders[self.couplesSize:]
            
            ### Cross-Over
            for m,f in zip(males,females):
                s1,s2 = self.cross_over(m,f)
                schedulesPopulation_new.append(s1)
                schedulesPopulation_new.append(s2)
            
            ### Goal Test [2]
            goalResult = self.goalTest(program, schedulesPopulation)
            if(goalResult!=-1):
                return goalResult,'Genetic Algorithm Succeeded (Gen {})'.format(iteration)
            
            ### Mutation
            for schedule in schedulesPopulation_new:
                #Probability of mutation happening
                if(random.randint(0,100)<10):
                    #Update individual after mutation
                    schedule = self.mutation(schedule)
                    
            ### Update population with better individuals
            schedulesPopulation = schedulesPopulation_new
        return 'Genetic Algorithm Failed'
    
    def goalTest(self, program, schedulePopulation):
        # run fitness method
        self.fitnessResults = list(self.fitness(program,schedule) for schedule in schedulePopulation)
        #print(list('%.1f' % elem for elem in self.fitnessResults))
        try:
            goalIndex = self.fitnessResults.index(1)
            return schedulePopulation[goalIndex]
        except(ValueError):
            return -1
    
    def fitness(self, program, schedule):
        tCollisionCount = 0.0
        # walk into week (mon-fri) and classtime (AB-CD)
        for week in range(len(schedule[0])):
            for classTime in range(len(schedule[0][0])):
                # each position to compare
                for s_out in range(len(schedule)):
                    for s_in in range(s_out+1,len(schedule)-1):
                        v_out = schedule[s_out][week][classTime]
                        v_in  = schedule[s_in][week][classTime]
                        if(v_out != None and v_in != None):
                            # verify if has collision
                            if(v_in.teacher.id==v_out.teacher.id):
                                # increment
                                tCollisionCount+=1
        # return fitness = 1 - (collision)/(maxCollision)
        return (1 - (tCollisionCount/program.tCollisionMax))
    
    def selection(self, schedulePopulation):
        selected = []
        # select 2*couple individuals
        for i in range(2*self.couplesSize):
            ### Using Addicted Roulette to select each individual
            # sum fitness results
            fitnessResSum = sum(self.fitnessResults)
            # separate roulette portions
            roulettePortions = list(f/fitnessResSum for f in self.fitnessResults)
            # set intervals
            intervals = []
            accum = 0
            for rp in roulettePortions:
                accum+=rp
                intervals.append(round(accum,2))
            # spin addicted roulette
            rouletteResult = random.randint(0,len(schedulePopulation))/len(schedulePopulation)
            # get selected indivitual based on the addicted roulette result
            individual = 0
            while(rouletteResult>intervals[individual]):
                individual+=1
            # append on select list
            selected.append(schedulePopulation[individual])
        # return selected list
        return selected
    
    def cross_over(self, male, female):
        # create copies of male and female
        m   = copy.deepcopy(male)
        f   = copy.deepcopy(female)
        # get a random number of cut
        cut = random.randint(0,len(m)-1)
        # cross-over male and female
        male.schedule   = m.schedule[:cut] + f.schedule[cut:]
        female.schedule = f.schedule[:cut] + m.schedule[cut:]
        # return male and female
        return male,female
    
    def mutation(self, schedule):
        # select semester to mutate
        semesterToMutate = random.randint(0,len(schedule)-1)
        geneShift        = random.randint(1,WEEK_CLASSES_SIZE-1)
        # get sequential week numbers
        seqWeek = []
        for week in range(len(schedule[0])):
            for classTime in range(len(schedule[0][0])):
                seqWeek.append(schedule[semesterToMutate][week][classTime])
        # shift (geneShift times) the seqWeek list
        for i in range(geneShift):
            last = seqWeek.pop()
            seqWeek.insert(0, last)
        # update shifted list on schedule
        cont = 0
        for week in range(len(schedule[0])):
            for classTime in range(len(schedule[0][0])):
                schedule[semesterToMutate][week][classTime] = seqWeek[cont]
                cont+=1
        # return schedule
        return schedule

In [6]:
while True:
    GA = GeneticAlgorithm_ProgramSchedule(200,20)
    solution = GA.geneticRun(engComp)
    if(solution!='Genetic Algorithm Failed'):
        print(solution)
        break

(|  Seg  Ter  Qua  Qui  Sex  |
|---------------------------|
|        Semester 1         |
|---------------------------|
|   1    2    2    4    3   | AB
|   3    3    4    2    1   | CD
|---------------------------|
|        Semester 2         |
|---------------------------|
|   8    7    6    9    6   | AB
|   9    7    5    8    5   | CD
|---------------------------|
|        Semester 3         |
|---------------------------|
|  11   13   14   10   12   | AB
|  14   11   10   12   13   | CD
|---------------------------|
|        Semester 4         |
|---------------------------|
|  17   18   16   17   15   | AB
|  18   17   15   18   16   | CD
|---------------------------|
|        Semester 5         |
|---------------------------|
|  20    0   20   21   22   | AB
|  23   19   21   22   23   | CD
|---------------------------|
|        Semester 6         |
|---------------------------|
|  26   25   24   26   24   | AB
|  25   27   27   27    0   | CD
|---------------------------|
|  